In [1]:
import numpy as np
import tensorflow as tf
import torch
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from dataloader import get_dataloaders
import numpy as np
import warnings
from tqdm import tqdm
from time import time
import json
from sklearn.metrics import precision_recall_fscore_support
import dni
warnings.filterwarnings("ignore")


In [2]:

def one_hot(indexes, n_classes):
    result = torch.FloatTensor(indexes.size() + (n_classes,))
    result = result.cpu()
    result.zero_()
    indexes_rank = len(indexes.size())
    result.scatter_(
        dim=indexes_rank, index=indexes.data.unsqueeze(dim=indexes_rank), value=1
    )
    return Variable(result)


class Four_Layer_SG(nn.Module):
    """4-layer CNN as described in the paper"""

    def __init__(self):
        super(Four_Layer_SG, self).__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=32,
                kernel_size=(12, 12),
                stride=(4, 4),
                padding=(4, 4),
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(12 * 12 * 32, 512),
            nn.ReLU(True),
            nn.Dropout(p=0.65),
            nn.Linear(512, 10),
        )

        self.backward_interface_1 = dni.BackwardInterface(
            dni.BasicSynthesizer(output_dim=12, n_hidden=1)
        )

    def forward(self, x, y=None):
        x = self.block_1(x)
        if self.training:
            # context = one_hot(y, 10)
            # with dni.synthesizer_context(context):
            x = self.backward_interface_1(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)


def test(model, test_loader):
    test_metrics = []

    model.eval()
    model.cuda()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()

            pred = output.data.max(1, keepdim=True)[
                1
            ]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            total += target.size(0)

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / total

    prec, recall, f1, support = precision_recall_fscore_support(
        target.cpu(), pred.cpu(), average="weighted"
    )

    test_metrics.append((test_accuracy.item(), prec, recall, f1, support))

    print(
        "\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), test_accuracy
        )
    )
    return test_metrics


def train_sg(model, epoch, train_loader, log_interval, verbose=False):
    model.train()
    model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=3e-5)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data, target)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if verbose:
            if batch_idx % log_interval == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(data),
                        loss.data.item(),
                    )
                )

In [6]:
torch.cuda.device_count()

2

In [3]:

n_epochs = 10000
log_interval = 20

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

batch_size_train = 256  # specified in the paper
train_loader, test_loader = get_dataloaders(1000, 200, batch_size_train)

model_dict = {
    "Four_Layer_SG": Four_Layer_SG,
}

# with open("data_SG.json", "r") as fp:
#     data = json.load(fp)

for key in model_dict.keys():
    model = model_dict[key]()
    torch.nn.DataParallel(model)
    stats = []
    test(model, test_loader)
    print(key, "\n")
    for epoch in tqdm(range(1, n_epochs + 1), leave=False):
        start_time = time()

        train_sg(model, epoch, train_loader, log_interval, verbose=False)

        end_time = time()
        time_taken = end_time - start_time

        metrics = test(model, test_loader)
        metrics.append(time_taken)

        stats.append(metrics)

    data[key] = stats
    # clear gpu memory
    torch.cuda.empty_cache()

# with open("data_SG.json", "w") as fp:
#     json.dump(data, fp)

  0%|                                                                                                                                                                                       | 0/10000 [00:00<?, ?it/s]


Test set: Avg. loss: 2.3177, Accuracy: 17/200 (8%)

Four_Layer_SG 



  0%|                                                                                                                                                                            | 1/10000 [00:03<10:57:49,  3.95s/it]


Test set: Avg. loss: 2.2386, Accuracy: 65/200 (32%)



  0%|                                                                                                                                                                             | 2/10000 [00:04<4:56:53,  1.78s/it]

...


Test set: Avg. loss: 0.2784, Accuracy: 182/200 (91%)



  1%|▉                                                                                                                                                                             | 51/10000 [00:16<40:35,  4.09it/s]


Test set: Avg. loss: 0.2798, Accuracy: 183/200 (92%)




Test set: Avg. loss: 0.2758, Accuracy: 183/200 (92%)



KeyboardInterrupt: 